# <center>Plotting heatmap from Excel for ChIP data</center>

This is a Jupyter notebook that you can use to plot interactively 
a heatmap from a sheet of an excel file that looks like this 

![](heatmap_chip_sheet.png)

The exact header and row index labels are not important.

You will need to rename the sheet you want to plot sns_*, e.g. sns_mcm or sns_pcna

In [23]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

%matplotlib inline

# Set Arial 12 pt as default font for plotting and embedd font in file 
# when exporting to pdf

plt.rc('font', family='Arial', size = 12)
plt.rc('pdf', fonttype=42)

def get_excel_files():
    """Get all .xlsx files from current directory"""
    
    excel_file_list = []
    for file in os.listdir(os.getcwd()):
        if file.lower().endswith(".xlsx"):
            excel_file_list.append(os.path.join(os.getcwd(), file))
    return ['Choose a file'] + excel_file_list

def get_excel_sheet_names(excel_file):
    """Get all sheets that start with sns_ in an excel file"""
    return [sheet_name for sheet_name in pd.ExcelFile(excel_file).sheet_names if sheet_name.startswith('sns_')]

file_widget = widgets.Dropdown(
    options=get_excel_files(),
    description='File name:',
    disabled=False) 

sheet_widget = widgets.Dropdown(
    description='Sheet name:',
    disabled=False)

def update_sheet_range(*args):
    """Update list of sheets options based on selected escel file"""
    sheet_widget.options = get_excel_sheet_names(file_widget.value)
file_widget.observe(update_sheet_range, 'value')

split_column_widget = widgets.IntSlider(
    min=0,
    step=1,
    description='Split by col #:',
    disabled=False,
    orientation='horizontal')

scale_choice_widget = widgets.Checkbox(
    value=False,
    description='Same scale?',
    disabled=False)

position_x_labels_widget = widgets.Dropdown(
    options=["top", "bottom"],
    description='x labels pos.:',
    disabled=False) 

save_choice_widget = widgets.ToggleButton(
    value=False,
    description='Save plot',
    disabled=False,
    button_style='',
    tooltip='Save plot as pdf')

def update_column_range(*args):
    """Set max of split by column slider to # of columns selected sheet/dataframe"""
    try:
        split_column_widget.max = len(pd.read_excel(file_widget.value, sheet_name=get_excel_sheet_names(file_widget.value)[0]).columns) - 1
    except IndexError:
        pass
        
file_widget.observe(update_column_range, 'value')

cmap_widget = widgets.Dropdown(
    options=[ 'YlGnBu', 'OrRd',
            'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
            'YlOrBr', 'YlOrRd', 'PuRd', 'RdPu', 'BuPu',
            'GnBu', 'PuBu', 'PuBuGn', 'BuGn', 'YlGn'],
    description='Colour map:',
    disabled=False) 

def plot_one_heatmap(input_df, labels, vmax, ax, cmap, position_y_labels, position_x_labels):
    """Plot heatmap"""
    
    divider = make_axes_locatable(ax)
    
    cax_width = str(50/len(input_df.columns)) + "%"
    
    if position_y_labels == "left":
        cax = divider.append_axes("right", size=cax_width, pad=0.05)
    else:
        cax = divider.append_axes("left", size=cax_width, pad=0.05)
    sns.heatmap(input_df, cmap=cmap, square=True, ax=ax, vmin = 0, vmax=vmax,
                        linewidths=0, cbar_ax = cax)
    if position_y_labels == "left":
        ax.yaxis.tick_left()
    else:
        ax.yaxis.tick_right()
        cax.yaxis.tick_left() 
    if position_x_labels == "top":
        ax.xaxis.tick_top() 
    ax.set_ylabel(labels.upper(), fontsize=14, fontweight='bold')
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none') 
    ax.tick_params('y', rotation = 0)
    ax.tick_params('x', rotation = 90)

def plot_heatmap(excel_file, sheet, cmap, split_column, scale_choice, position_x_labels, save):
    """Plot the desired type of heatmap"""
    
    try:
        input_df = pd.read_excel(excel_file, sheet_name=sheet, index_col=0)
        label = sheet.split("_")[1]
                
        if split_column == 0:
            fig, ax = plt.subplots(1,1, figsize=(5,3.5))
            plot_one_heatmap(input_df, label, None, ax, cmap, "left", position_x_labels)
        else:
            fig, ax = plt.subplots(1,2, figsize=(10,3.5))
            if not scale_choice:
                vmax = None
            else:
                vmax = input_df.max().max()
            plot_one_heatmap(input_df.iloc[:,:split_column], label, vmax, ax[0], cmap, "left", position_x_labels)
            plot_one_heatmap(input_df.iloc[:,split_column-1:], "", vmax, ax[1], cmap, "right", position_x_labels)
        plt.tight_layout()
        if save:
            fig.savefig(excel_file.split('.')[0] + "_" + label + ".pdf")
            save_choice_widget.value = False
    except IOError:
        print("Cannot save PDF because a file by the same name is already open")
    except AttributeError:
        print("The Excel file you selected doesn't seem right")
    except:
        print("Select a valid file and sheet")
        
x=interactive(plot_heatmap, 
              excel_file=file_widget,
              sheet=sheet_widget,
              cmap = cmap_widget,
              split_column = split_column_widget,
              scale_choice=scale_choice_widget,
              position_x_labels = position_x_labels_widget,
              save=save_choice_widget)
display(x)

interactive(children=(Dropdown(description='File name:', options=('Choose file', '/mnt/c/Users/niczilio/OneDri…